In [2]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Load the dataset from the 'data' sub-folder
df = pd.read_csv('data/CFB_TransferPortal23.csv', encoding='ISO-8859-1')  # Added the encoding based on our previous discovery

# Create the Dash app
app = dash.Dash(__name__)
server = app.server

app.layout = html.Div([
    html.Div([
        html.H1("College Football Transfer Portal Analysis", style={'display': 'inline-block'}),
        # You can add an image here if you have one
    ]),
    
    html.P(
        "This dashboard visualizes the trends in college football transfers for 2023.",
        style={"margin": "20px"}),
    
    html.Div([
        dcc.Dropdown(
            id='data-dropdown',
            options=[
                {'label': 'By Conference', 'value': 'Conference'},
                {'label': 'By Receiving Team', 'value': 'Receiving Team'},
                {'label': 'By Position', 'value': 'Position'}
            ],
            value='Conference',
            style={'width': '50%'}
        ),
        dcc.Graph(id='data-graph')
    ]),
])

@app.callback(
    Output('data-graph', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_graph(selected_value):
    if selected_value == 'Conference':
        data = df['Conference'].value_counts()
        title = "Number of Transfers by Conference"
    elif selected_value == 'Receiving Team':
        data = df['Receiving Team'].value_counts()
        title = "Top Teams with the Most Transfers"
    else:
        data = df['Position'].value_counts()
        title = "Number of Transfers by Position"
    
    fig = px.bar(data, title=title, labels={'value': 'Number of Transfers', 'index': selected_value})
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)


In [7]:
import pandas as pd
import plotly as go
import plotly.graph_objects as go

# Reload the dataset since the environment was reset
df = pd.read_csv('C:/Users/Ben/OneDrive/Documents/CFB_TransferPortal/data/CFB_TransferPortal23.csv', encoding='ISO-8859-1')

# Filter the dataset for 'Power 5' conference type
power5_df = df[df['Conference'] == 'SEC']

# Group by 'From School' and 'Receiving Team' and count transfers
grouped = power5_df.groupby(['From School', 'Receiving Team']).size().reset_index(name='count')

# Extract unique schools and teams for nodes
all_nodes = list(set(grouped['From School']).union(set(grouped['Receiving Team'])))

# Map 'From School' and 'Receiving Team' to their indices in the node list
grouped['source'] = grouped['From School'].apply(lambda x: all_nodes.index(x))
grouped['target'] = grouped['Receiving Team'].apply(lambda x: all_nodes.index(x))

# Create the Sankey diagram
sankey_fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes
    ),
    link=dict(
        source=grouped['source'],
        target=grouped['target'],
        value=grouped['count']
    ))])

sankey_fig.update_layout(title_text="Transfers from 'From School' to 'Receiving Team' (Power 5)", font_size=10)
sankey_fig
